<a href="https://colab.research.google.com/github/Fetrixdu83/VBA_Project/blob/main/UE05_Fintech_Md%20Jasam%20Sikder%20FIT%202025-26.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install python-bitcoin-utils

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.5/872.5 kB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 658.2/658.2 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.7/285.7 kB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 68.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 7.7 MB/s eta 0:00:00
  Created wheel for python-bitcoinrpc: filename=python_bitcoinrpc-1.0-py3-none-any.w

In [ ]:
from bitcoinutils.keys import PrivateKey
priv = PrivateKey()          # random testnet private key
print("WIF:", priv.to_wif())
print("address:", priv.get_public_key().get_address().to_string())
# IMPORTANT: don't share priv.to_wif() or seed

WIF: cQadi8Yz3qzfCMPyAwcBHYiA7wdr9Wq3H7eDZHpUwFWPyFsDXMDE
address: mphvk9fMV3o9yZvwZqkTgzBAUv8nnoz7J4


In [ ]:
!pip install bitcoin-utils
from bitcoinutils.setup import setup
from bitcoinutils.keys import PrivateKey, P2pkhAddress
from bitcoinutils.transactions import Transaction, TxInput, TxOutput
from bitcoinutils.script import Script

WIF = "cQadi8Yz3qzfCMPyAwcBHYiA7wdr9Wq3H7eDZHpUwFWPyFsDXMDE"

# Your unspent outputs (add as many as needed)
UTXOS = [
    {"txid": "410adc05622fefdb81ac13205000dd23540911bf17b56b22b03cbac9f15563e7", "vout": 0, "value": 191099},   # example sats
]

SEND_ADDR = "mreLpAzPWBtdwBC9NMEsBy1jkQ3phjy1Eh"
SEND_SATS = 100_000
FEE_SATS  = 1_000  # safe small fee on testnet
# ----------------------------------

setup('testnet')

priv = PrivateKey.from_wif(WIF)
pub  = priv.get_public_key()
my_change_addr = pub.get_address()  # P2PKH change address (same key)

# build inputs
inputs = [TxInput(u["txid"], u["vout"]) for u in UTXOS]
total_in = sum(u["value"] for u in UTXOS)

# check funding
needed = SEND_SATS + FEE_SATS
assert total_in >= needed, f"Insufficient funds: have {total_in} sats, need {needed} sats"

change_sats = total_in - needed

# outputs: to teacher + change back to you (if any)
outs = [TxOutput(SEND_SATS, P2pkhAddress(SEND_ADDR).to_script_pub_key())]
if change_sats > 0:
    outs.append(TxOutput(change_sats, my_change_addr.to_script_pub_key()))

# legacy (non-SegWit) P2PKH
tx = Transaction(inputs, outs, has_segwit=False)

# sign each input against your own P2PKH scriptPubKey
prev_spk = my_change_addr.to_script_pub_key()
for i in range(len(inputs)):
    sig = priv.sign_input(tx, i, prev_spk)
    inputs[i].script_sig = Script([sig, pub.to_hex()])  # <sig> <pubkey>

raw_hex = tx.serialize()
txid = tx.get_txid()
print("RAW:", raw_hex)
print("TXID:", txid)

RAW: 0200000001e76355f1c9ba3cb0226bb517bf11095423dd00502013ac81dbef2f6205dc0a41000000006a47304402202f1f2094a807adb440f3b594bc26ed871442ddb3e5e1ef2a12e13a59b3e9fd1802202f85e8e70b7cbdaabc3a205f0568efd89478e966c218452ccee4eb89e62a0c3601210311cd0f730150a8ff16a17095f2899e148c45a5ec86aed58ca4826365c606eb8afdffffff02a0860100000000001976a9147a0f17f08359d9bff71ad97f1eb39b694ad67c0488acf35f0100000000001976a91464cc69e87268b902930cd812cece94261cacc53488ac00000000
TXID: e86f752d737937efc664ecd1382f15b1667403c7712bcfa0db6d6c9541bcbeac


In [ ]:
from bitcoinutils.setup import setup
from bitcoinutils.keys import PrivateKey
from bitcoinutils.transactions import Transaction, TxInput, TxOutput
from bitcoinutils.script import Script

setup('testnet')

# ---------- YOUR DATA ----------
WIF = "cQadi8Yz3qzfCMPyAwcBHYiA7wdr9Wq3H7eDZHpUwFWPyFsDXMDE"                      # keep secret!
UTXOS = [                                          # copy from explorer
    {"txid": "e86f752d737937efc664ecd1382f15b1667403c7712bcfa0db6d6c9541bcbeac", "vout": 1, "value": 90099}
]
SEND_SATS = 50_000                                # 0.0005 tBTC
FEE_SATS  = 1000
TEACHER_PUB = "02019662a808d4a0df7e8c1ee8b26646e59cfaa92ebd906bde14b4bda5113fa2a9"
# --------------------------------

# Your keypair & change address (P2PKH)
priv = PrivateKey.from_wif(WIF)
pub  = priv.get_public_key()
change_spk = pub.get_address().to_script_pub_key()

# Build the bare 2-of-2 multisig *locking script*
#   2 <YourPubKey> <TeacherPubKey> 2 OP_CHECKMULTISIG
multisig_spk = Script(['OP_2', pub.to_hex(), TEACHER_PUB, 'OP_2', 'OP_CHECKMULTISIG'])

# Inputs / totals
inputs = [TxInput(u["txid"], u["vout"]) for u in UTXOS]
total_in = sum(u["value"] for u in UTXOS)
assert total_in >= SEND_SATS + FEE_SATS, "Insufficient funds"

change_sats = total_in - SEND_SATS - FEE_SATS

# Outputs: multisig + change
outs = [TxOutput(SEND_SATS, multisig_spk)]
if change_sats > 0:
    outs.append(TxOutput(change_sats, change_spk))

# Create a *legacy* tx; your inputs are standard P2PKH
tx = Transaction(inputs, outs, has_segwit=False)

# Sign each input against your previous output's P2PKH script
for i in range(len(inputs)):
    sig = priv.sign_input(tx, i, change_spk)
    inputs[i].script_sig = Script([sig, pub.to_hex()])   # <sig> <pubkey>

raw_hex = tx.serialize()
txid = tx.get_txid()
print("RAW HEX:", raw_hex)
print("TXID:", txid)


RAW HEX: 0200000001acbebc41956c6ddba0cf2b71c7037466b1152f38d1ec64c6ef3779732d756fe8010000006a47304402201410c92801a3880abab8cebbf79d024cf2094a9f329454577f7058a8bd3a21c802206add193a41bcfac5ce4fabc9bcde690f4b7fc3d3a79057a26f0584b6d19498c201210311cd0f730150a8ff16a17095f2899e148c45a5ec86aed58ca4826365c606eb8afdffffff0250c30000000000004752210311cd0f730150a8ff16a17095f2899e148c45a5ec86aed58ca4826365c606eb8a2102019662a808d4a0df7e8c1ee8b26646e59cfaa92ebd906bde14b4bda5113fa2a952aebb980000000000001976a91464cc69e87268b902930cd812cece94261cacc53488ac00000000
TXID: 4c21f8c5a5eb955ad46bace7adaa73b5d8f1b3fd97cf4c417fb32df481d640a3


In [ ]:
from bitcoinutils.setup import setup
from bitcoinutils.keys import PrivateKey
setup('testnet')

WIF = "cQadi8Yz3qzfCMPyAwcBHYiA7wdr9Wq3H7eDZHpUwFWPyFsDXMDE"
addr_from_wif = PrivateKey.from_wif(WIF).get_public_key().get_address().to_string()
print("Address from WIF:", addr_from_wif)


Address from WIF: mphvk9fMV3o9yZvwZqkTgzBAUv8nnoz7J4


In [ ]:
!pip install bitcoin-utils
from bitcoinutils.setup import setup
from bitcoinutils.keys import PrivateKey

setup('testnet')
priv = PrivateKey.from_wif("cQadi8Yz3qzfCMPyAwcBHYiA7wdr9Wq3H7eDZHpUwFWPyFsDXMDE")
print("address:", priv.get_public_key().get_address().to_string())
print("pubkey (hex):", priv.get_public_key().to_hex())


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 872.5/872.5 kB 34.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 658.2/658.2 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.7/285.7 kB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 kB 7.4 MB/s eta 0:00:00
  Created wheel for python-bitcoinrpc: filename=python_bitcoinrpc-1.0-py3-none-any.w

In [ ]:
from bitcoinutils.setup import setup
from bitcoinutils.keys import PrivateKey, P2pkhAddress
from bitcoinutils.transactions import Transaction, TxInput, TxOutput
from bitcoinutils.script import Script

setup('testnet')

WIF       = "cQadi8Yz3qzfCMPyAwcBHYiA7wdr9Wq3H7eDZHpUwFWPyFsDXMDE"
Q1_TXID   = "4c21f8c5a5eb955ad46bace7adaa73b5d8f1b3fd97cf4c417fb32df481d640a3"      # tx that created the multisig output
Q1_VOUT   = 1                         # the output index (#0/#1) of that multisig
Q1_VALUE  = 50000                    # value (sats) of that multisig UTXO
FEE_SATS  = 1000                      # choose a small fee
TEACHER_DEST = "msfTfNj6FicTNBShCJBhoxvhHoM794cKsZ"

TEACHER_PUB = "02019662a808d4a0df7e8c1ee8b26646e59cfaa92ebd906bde14b4bda5113fa2a9"
# ----------------------

priv = PrivateKey.from_wif(WIF)
pub  = priv.get_public_key()

# The **exact same locking script** you used in Q1:
# 2 <YourPub> <TeacherPub> 2 OP_CHECKMULTISIG
multisig_spk = Script(['OP_2', pub.to_hex(), TEACHER_PUB, 'OP_2', 'OP_CHECKMULTISIG'])

# Build the spend: one input (your multisig UTXO) → one output to teacher P2PKH
inp = TxInput(Q1_TXID, Q1_VOUT)
send_sats = Q1_VALUE - FEE_SATS
out = TxOutput(send_sats, P2pkhAddress(TEACHER_DEST).to_script_pub_key())

tx = Transaction([inp], [out], has_segwit=False)

# Produce **your** signature over input 0 using the multisig script as the prevout script
my_sig = priv.sign_input(tx, 0, multisig_spk)

# IMPORTANT: Do NOT set scriptSig here → keep it unsigned for the teacher
unsigned_hex = tx.serialize()

print("UNSIGNED_TX_HEX:", unsigned_hex)
print("MY_SIGNATURE_HEX:", my_sig)


UNSIGNED_TX_HEX: 0200000001a340d681f42db37f414ccf97fdb3f1d8b573aaade7ac6bd45a95eba5c5f8214c0100000000fdffffff0168bf0000000000001976a914853d5a191fd2ab19ebbd9519a696e5f30f7d119788ac00000000
MY_SIGNATURE_HEX: 304402204d3c9fe7a0b376078d9a4f8a966d2aa5badb9f2216390d0b7baaf1b58401cf04022063cbf78f1a71e12e3b92fa775542b515435fb5c6a839f52e786b6e905d2e1adb01
